In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn import metrics

In [2]:
df=pd.read_csv("D:\AI-PROJECT\Student Performance Analysis\Student Performance Analysis\Copy_students.csv")

In [3]:
df['final_grade'] = 'na'
df.loc[(df.G3 >= 15) & (df.G3 <= 20), 'final_grade'] = 0
df.loc[(df.G3 >= 8) & (df.G3 <= 14), 'final_grade'] = 1 
df.loc[(df.G3 >= 0) & (df.G3 <= 7), 'final_grade'] = 2 
df.head(5)

,sex,age,Medu,Fedu,Mjob,studytime,failures,schoolsup,paid,higher,internet,romantic,goout,health,absences,G1,G2,G3,subject,final_grade
0,0,18,4,4,4,2,0,1,0,1,0,0,4,3,4,0,11,11,0,1
1,0,17,1,1,4,2,0,0,0,1,1,0,3,3,2,9,11,11,0,1
2,0,15,1,1,4,2,0,1,0,1,1,0,2,3,6,12,13,12,0,1
3,0,15,4,2,1,3,0,0,0,1,1,1,2,5,0,14,14,14,0,1
4,0,16,3,3,5,2,0,0,0,1,0,0,2,5,0,11,13,13,0,1


In [4]:
dfd = df.copy()
dfd = dfd.drop(['G3'], axis=1)

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
dfd.final_grade = le.fit_transform(dfd.final_grade)

In [6]:
from sklearn.model_selection import train_test_split
X = dfd.drop('final_grade',axis=1)
y = dfd.final_grade
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)

In [7]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [8]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sf = sgd.fit(X_train, y_train)
print("Stochastic Gradient Descent Model Score" , ":" , sf.score(X_train, y_train) , "," ,
      "Cross Validation Score" ,":" , sf.score(X_test, y_test))

Stochastic Gradient Descent Model Score : 0.8410958904109589 , Cross Validation Score : 0.8248407643312102


In [11]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population):
    scores = []
    for chromosome in population:
        sgd.fit(X_train.iloc[:,chromosome],y_train)
        predictions = sgd.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,X_test, y_train, y_test):
                                   
    best_chromo= []
    best_score= []
    population_nextgen = initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [12]:
sgd = SGDClassifier()
chromo,score=generations(size=100,n_feat=18,n_parents=5,mutation_rate=0.20, n_gen=5,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)
sgd.fit(X_train.iloc[:,chromo[-1]],y_train)
predictions = sgd.predict(X_test.iloc[:,chromo[-1]])
#"Accuracy score after genetic algorithm is= "+
print( str(accuracy_score(y_test,predictions)))

C:\Users\Admin\AppData\Local\Temp/ipykernel_19668/385242641.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  chromosome = np.ones(n_feat,dtype=np.bool)


[0.910828025477707, 0.9076433121019108]
[0.8789808917197452, 0.8789808917197452]
[0.8694267515923567, 0.8630573248407644]
[0.9267515923566879, 0.9171974522292994]
[0.9012738853503185, 0.8949044585987261]
0.8757961783439491
